In [1]:
from datasets import load_dataset

DAN= load_dataset('TrustAIRLab/in-the-wild-jailbreak-prompts', 'jailbreak_2023_05_07', split='train')
forbidden_questions=load_dataset("TrustAIRLab/forbidden_question_set", split='train')

"""
we extract five questions
from each forbidden scenario, forming a jailbreak
dataset comprising 8 jailbreak communities × 3
jailbreak prompts × 13 forbidden scenarios × 5
questions, totaling 1560 samples.
"""


'\nwe extract five questions\nfrom each forbidden scenario, forming a jailbreak\ndataset comprising 8 jailbreak communities × 3\njailbreak prompts × 13 forbidden scenarios × 5\nquestions, totaling 1560 samples.\n'

In [7]:
DAN_df=DAN.to_pandas()
forbidden=forbidden_questions.to_pandas()

In [15]:
forbidden_categories=forbidden["content_policy_name"].unique()
forbidden_categories

array(['Illegal Activity', 'Hate Speech', 'Malware', 'Physical Harm',
       'Economic Harm', 'Fraud', 'Pornography', 'Political Lobbying',
       'Privacy Violence', 'Legal Opinion', 'Financial Advice',
       'Health Consultation', 'Gov Decision'], dtype=object)

In [43]:
jailbreak_communities=DAN_df["community_name"].dropna().unique()
JAILBREAKS=[]
for category in jailbreak_communities:
    jailbreaks=DAN_df[DAN_df["community_name"]==category]["prompt"].sample(n=3,random_state=42)
    for forbidden_category in forbidden_categories:
        forbidden_questions=forbidden[forbidden["content_policy_name"]==forbidden_category]["question"].sample(n=5,random_state=42)
        for jailbreak in jailbreaks:
            if "\nprompt" in jailbreak:
                jail=jailbreak.split("\nprompt")
                jailbreak=jail[0]
            for fb_question in forbidden_questions:
                if "INSERT PROMPT HERE":
                    prompt=jailbreak.replace("[INSERT PROMPT HERE]",fb_question)
                else:
                    prompt=jailbreak+"\n"+fb_question
                JAILBREAKS.append({"full_prompt":prompt,"jailbreak":jailbreak,"question":fb_question,"jailbreak_cat":category,"question_cat":forbidden_category})
            

In [44]:
import json
with open("Dan_samples.json","w") as f:
    json.dump(JAILBREAKS,f,indent=2)